In [1]:
for p in ("Knet", "Plots", "NBInclude")
    Pkg.installed(p) == nothing && Pkg.add(p);
end

In [2]:
using Knet, Plots, NBInclude;
nbinclude("deepppiutils.ipynb");

In [3]:
featuresDict = constructFeatDictNegatome();
concatAB, ygold = loaddataNegatome(featuresDict);

In [ ]:
# the number of hidden units in the hidden layers of the DeepPPI-sep model
HIDDENSSEP = Any[NOINPUTS, 512, 256, 128]; 
HIDDENSMER = Any[256, 128, NOOUTPUTS]
NOEPOCH = 30;
BATCHSIZE = 50;
PDROP = (0, 0.2);

In [ ]:
accuracy =[]
recall=[]
specifity=[]
precision= []
mcc=[]
f1=[]
npv=[]
accuracyt= recalli=specifityi=precisioni=mcci = 0.0
for i in 1:5
    #setseed(i);
    wa = winit(HIDDENSSEP...);
    wb = winit(HIDDENSSEP...);
    wMerged = winit(HIDDENSMER...);
    w = vcat(wa, wb, wMerged);
    
    #dtrn, ddev, dtst = dividedataset(concatAB, ygold, trnper, devper, tstper; batchsize= BATCHSIZE);
    dtrn, dtst = dividedatasetNegatome(concatAB, ygold, trnper, devper, tstper; batchsize= BATCHSIZE, dev=false);
    
    optims = params(w; optim="Momentum", lr=0.01, gamma=0.9);
    #@time trnloss, trnerr, tstloss, tsterr=trainSep!(w, optims, dtrn, predictSep, ddev; pdrop=PDROP, epochs=NOEPOCH) 
    @time trainSep!(w, optims, dtrn, predictSep, dtst; pdrop=PDROP, epochs=NOEPOCH) 
    
    println("Dataset", i)
    println("Training: min. loss =",loss(w,dtrn,predictSep),", min. error =",zeroone(w,dtrn,predictSep))  
    println("Test: min. loss =",loss(w,dtst,predictSep),", min. error =",zeroone(w,dtst,predictSep))  
    
    accuracyt,recalli,specifityi,precisioni,mcci,f1i,npvi = modelevaluation(w, dtst, predictSep; p=true);
    push!(accuracy, accuracyt)
    push!(recall, recalli)
    push!(specifity, specifityi)
    push!(precision, precisioni)
    push!(mcc, mcci)
    push!(f1, f1i)
    push!(npv, npvi)
    
    writedlm("DeepPPI_SepModel"*string(i)*".csv", map(Array, w))
end

In [ ]:
#summary(accuracy)
open("DeepPPI-Sep_Scores_wNegatome2.txt", "w") do f
    write(f, "Dataset \tAccuracy\t\t\tPrecision\t\t\tnpv      \t\t\tRecall   \t\t\tSpecifity\t\t\tMCC\n")
    for i in 1:5
        write(f, "dataset"*string(i)*"\t"*string(accuracy[i]) *"\t"* string(precision[i]) *"\t"* string(npv[i]) *"\t"* string(recall[i]) *"\t"*  string(specifity[i]) *"\t"*  string(mcc[i]) *"\n")
    end
    write(f, "Average"*"\t\t"*string(mean(accuracy)) *"\t"* string(mean(precision))  *"\t"* string(mean(npv)) *"\t"* string(mean(recall)) *"\t"*  string(mean(specifity)) *"\t"*  string(mean(mcc)) *"\n")
end;
#println("Accuracy", "   Precision", "   npv"," recall", "    specifity", "     mcc", "       f1")
#(hcat(accuracy, precision, npv, recall, specifity, mcc, f1))

# DeepPPI-Con With Negatome Dataset

In [5]:
# the number of hidden units in the hidden layers of the DeepPPI-CON model
HIDDENS = Any[NOCONCAT, 512, 256, 128, 128, NOOUTPUTS]; 
NOEPOCH = 30;
BATCHSIZE = 50;
PDROP = (0, 0.2);

In [6]:
accuracy =[]
recall=[]
specifity=[]
precision= []
mcc=[]
f1=[]
npv=[]
accuracyt= recalli=specifityi=precisioni=mcci = 0.0
for i in 1:5
    #setseed(i);
    w = winit(HIDDENS...);
    
    #dtrn, ddev, dtst = dividedataset(concatAB, ygold, trnper, devper, tstper; batchsize= BATCHSIZE);
    dtrn, dtst = dividedatasetNegatome(concatAB, ygold, trnper, devper, tstper; batchsize= BATCHSIZE, dev=false);
    
    optims = params(w; optim="Momentum", lr=0.01, gamma=0.9);
    #@time trnloss, trnerr, tstloss, tsterr=train!(w, optims, dtrn, predictSep, ddev; pdrop=PDROP, epochs=NOEPOCH) 
    @time train!(w, optims, dtrn, predict, dtst; pdrop=PDROP, epochs=NOEPOCH) 
    
    println("Dataset", i)
    println("Training: min. loss =",loss(w,dtrn,predict),", min. error =",zeroone(w,dtrn,predict))  
    println("Test: min. loss =",loss(w,dtst,predict),", min. error =",zeroone(w,dtst,predict))  
    
    accuracyt,recalli,specifityi,precisioni,mcci,f1i,npvi = modelevaluation(w, dtst, predict; p=true);
    push!(accuracy, accuracyt)
    push!(recall, recalli)
    push!(specifity, specifityi)
    push!(precision, precisioni)
    push!(mcc, mcci)
    push!(f1, f1i)
    push!(npv, npvi)
    
    writedlm("DeepPPI_ConModel"*string(i)*".csv", map(Array, w))
end

12444×2328 Array{Float32,2}
(:epoch, 10, :trn, 0.9888770053475927, :tst, 0.9536507936507935)
(:epoch, 20, :trn, 0.9979679144385025, :tst, 0.9619047619047618)
(:epoch, 30, :trn, 0.9981818181818182, :tst, 0.9593650793650791)
162.850690 seconds (8.29 M allocations: 36.752 GiB, 4.56% gc time)
Dataset1
Training: min. loss =0.027462926, min. error =0.005989304812834995
Test: min. loss =0.26537022, min. error =0.04349206349206347
TP: 1481 , TN: 1493 , FP: 54 , FN: 83
Model evaluation:
Accuracy : 0.9559627126467494
Precision : 0.9648208462769896
NPV : 0.9473350247796097
Sensitivity / Recall : 0.9469309456861055
Specifity : 0.9650937291757635
MCC : 0.9120902718294054
F1 : 0.9557921907209448
12444×2328 Array{Float32,2}
(:epoch, 10, :trn, 0.9841711229946511, :tst, 0.9571428571428571)
(:epoch, 20, :trn, 0.996898395721925, :tst, 0.9593650793650793)
(:epoch, 30, :trn, 0.9988235294117647, :tst, 0.9628571428571429)
160.247836 seconds (6.71 M allocations: 36.673 GiB, 4.78% gc time)
Dataset2
Training: m

In [8]:
#summary(accuracy)
open("DeepPPI-Con_Scores_wNegatome2.txt", "w") do f
    write(f, "Dataset \tAccuracy\t\t\tPrecision\t\t\tnpv      \t\t\tRecall   \t\t\tSpecifity\t\t\tMCC\n")
    for i in 1:5
        write(f, "dataset"*string(i)*"\t"*string(accuracy[i]) *"\t"* string(precision[i]) *"\t"* string(npv[i]) *"\t"* string(recall[i]) *"\t"*  string(specifity[i]) *"\t"*  string(mcc[i]) *"\n")
    end
    write(f, "Average"*"\t\t"*string(mean(accuracy)) *"\t"* string(mean(precision))  *"\t"* string(mean(npv)) *"\t"* string(mean(recall)) *"\t"*  string(mean(specifity)) *"\t"*  string(mean(mcc)) *"\n")
end;
#println("Accuracy", "   Precision", "   npv"," recall", "    specifity", "     mcc", "       f1")
#(hcat(accuracy, precision, npv, recall, specifity, mcc, f1))